In [5]:
import httpx
from pebble import ThreadPool
from concurrent.futures import TimeoutError, as_completed
import time
import threading

def get_multiple(url: str, proxies: list):
    tic = time.time()
    
    # Local success flag - not global
    success_flag = threading.Event()
    
    def get_resp(url, proxy):
        # Check if another thread already succeeded
        if success_flag.is_set():
            print(f"Skipping {proxy} - already got success")
            return None
            
        try:
            with httpx.Client(proxy=proxy) as client:
                tic_req=time.time()
                print(f"Sending request via proxy: {proxy}")
                
                # Check again before making request
                if success_flag.is_set():
                    return None
                    
                response = client.get(url, timeout=30)
                
                # Check if we should even process this
                if success_flag.is_set():
                    return None
                    
                toc_req=time.time()
                print(f"Response time via proxy {proxy}: {toc_req-tic_req:.2f}s")
                print(f"Received response via proxy: {proxy} with status code {response.status_code}")
                
                # Return both status and content so we can check it
                return {
                    'status_code': response.status_code,
                    'content': response.text,
                    'proxy': proxy,
                    'time': toc_req-tic_req
                }
        except Exception as e:
            if not success_flag.is_set():
                print(f"Error with {proxy}: {str(e)}")
            return None
    
    pool = ThreadPool(max_workers=40)
    
    try:
        # Submit all tasks
        futures = [pool.schedule(get_resp, args=(url, proxy)) for proxy in proxies]
        
        # Use as_completed to get results as they finish (not in order!)
        for future in as_completed(futures):
            # If we already found success, break immediately
            if success_flag.is_set():
                break
                
            try:
                result = future.result(timeout=0.01)
                
                # Check if it's a successful response
                if result and isinstance(result, dict) and result.get('status_code') == 200:
                    toc = time.time()
                    print(f"\n✓ SUCCESS! Total time: {toc-tic:.2f}s")
                    print(f"✓ Successful response via proxy: {result['proxy']}")
                    
                    # Signal all other threads to stop
                    success_flag.set()
                    
                    # Return IMMEDIATELY - don't wait for anything
                    return result
                    
            except TimeoutError:
                pass
            except Exception as e:
                pass
                
    finally:
        # Close pool without waiting
        pool.close()
        
    return None


pp=['http://103.155.62.141:8081',
 'http://45.177.16.137:999',
 'http://190.242.157.215:8080',
 'http://187.102.219.64:999',
 'http://41.128.72.147:1981',
 'http://62.113.119.14:8080',
 'http://59.6.25.118:3128',
 'http://101.47.16.15:7890',
 'http://154.3.236.202:3128',
 'http://194.26.141.202:3128',
 'http://205.164.192.115:999']


world = "Auroria"
guildname = "Ascended Auroria"

url = f"https://rubinothings.com.br/guild.php?guild={guildname.replace(' ', '+')}&world={world}"

response = get_multiple(url, pp)
print(f"\nFinal response: {response}")


Sending request via proxy: http://62.113.119.14:8080Sending request via proxy: http://154.3.236.202:3128

Sending request via proxy: http://103.155.62.141:8081
Sending request via proxy: http://41.128.72.147:1981
Sending request via proxy: http://59.6.25.118:3128
Sending request via proxy: http://194.26.141.202:3128
Sending request via proxy: http://205.164.192.115:999
Sending request via proxy: http://101.47.16.15:7890
Sending request via proxy: http://45.177.16.137:999
Sending request via proxy: http://187.102.219.64:999
Sending request via proxy: http://190.242.157.215:8080
Response time via proxy http://154.3.236.202:3128: 1.52s
Received response via proxy: http://154.3.236.202:3128 with status code 200

✓ SUCCESS! Total time: 3.21s
✓ Successful response via proxy: http://154.3.236.202:3128

Final response: {'status_code': 200, 'content': '<!DOCTYPE html>\r\n<html lang="pt-br">\r\n<head>\r\n    <meta charset="UTF-8">\r\n<title>RubinOThings - Guild</title>\r\n<meta name="viewport" c